**Лабораторная работа № 1 «Классификация изображений с помощью сверточных нейронных сетей»**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image  import ImageDataGenerator
from keras import backend as K

Настраиваем параметрый для test и train данных

In [28]:
train_path = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training/"
test_path = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/"
shape_of_image = [100, 100, 3]
number_of_class = 131

Создаем источники данных для тренировки и валидации сети

In [29]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255, shear_range=0.3, horizontal_flip=True, zoom_range=0.3)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=shape_of_image[:2],
                                                    batch_size=32,
                                                    color_mode='rgb',
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=shape_of_image[:2],
                                                  batch_size=32,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


Создаем CNN и натраиваем ее

In [31]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu', input_shape = shape_of_image))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),activation = 'relu', input_shape = shape_of_image))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3),activation = 'relu', input_shape = shape_of_image))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(1024,activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(number_of_class,activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['acc',f1_m,precision_m, recall_m])

Обучаем модель

In [33]:
batch_size = 32
number_of_batch = 1600 // batch_size
hist = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = number_of_batch,
    epochs = 100,
    validation_data = test_generator,
    validation_steps = 800 // batch_size
)

Epoch 1/100
50/50 [==============================] - 15s 301ms/step - loss: 3.3054 - acc: 0.1713 - f1_m: 0.0474 - precision_m: 0.4027 - recall_m: 0.0256 - val_loss: 2.3759 - val_acc: 0.3975 - val_f1_m: 0.1506 - val_precision_m: 0.9169 - val_recall_m: 0.0838
Epoch 2/100
50/50 [==============================] - 15s 296ms/step - loss: 2.6167 - acc: 0.2975 - f1_m: 0.1468 - precision_m: 0.5528 - recall_m: 0.0875 - val_loss: 2.3068 - val_acc: 0.3550 - val_f1_m: 0.1959 - val_precision_m: 0.7280 - val_recall_m: 0.1175
Epoch 3/100
50/50 [==============================] - 15s 296ms/step - loss: 2.1042 - acc: 0.3850 - f1_m: 0.2807 - precision_m: 0.6397 - recall_m: 0.1831 - val_loss: 1.4169 - val_acc: 0.6100 - val_f1_m: 0.4846 - val_precision_m: 0.8497 - val_recall_m: 0.3425
Epoch 4/100
50/50 [==============================] - 15s 291ms/step - loss: 1.8122 - acc: 0.4744 - f1_m: 0.3809 - precision_m: 0.6722 - recall_m: 0.2700 - val_loss: 1.7748 - val_acc: 0.4700 - val_f1_m: 0.4020 - val_precision_m

Сохраняем веса

In [34]:
model.save_weights("trialv3.h5")